In [39]:
import numpy as np
from numpy import diff
from numpy import sum
from random import randint
import math
import time

import torch
print(torch.__version__)

import torch.nn as nn
import torch.nn.functional as F

np.__version__

2.2.0


'1.26.4'

In [ ]:
from enum import Enum

class Game_Moment(Enum):
    SECOS_1 = 1
    DADOS_1 = 2
    SECOS_2 = 3
    DADOS_2 = 4
    TABELA = 5
    FIM = 6

In [65]:
class Yan:
    def __init__(self):
        # iniciar a coluna da desordem com -1 para indicar que as celulas estão vazias
        self.desordem = {"1": -1, "2": -1, "3": -1, "4": -1, "5": -1, "6": -1, "q": -1, "f": -1, "s+": -1, "s-": -1, "x+": -1, "x-": -1, "y": -1}
        self.yangame = "YanGame"
        self.rolls_left = 2
        self.dices = [0,0,0,0,0]
        self.new_dice = list()
        self.marcado_em = ""
        self.over_minimum = False
        self.is_ended = False
        self.game_play = []
        self.next_state = self.get_game_state()
        self.reward = 0
        

        # self.action_size = self.row_count * self.column_count
    
    def check_consecutive(self, l):
        n = len(l) - 1
        return sum(diff(sorted(l)) == 1) >= n


    def roll_dice(self, n):
        rolls = list()
        for i in range(n):
            rolls.append(randint(1, 6))
        rolls.sort()
        self.game_play.append("você rolou:                          " + str(rolls))
        return rolls


    def get_game_state(self):
        state = [self.rolls_left]
        for die in self.dices:
            state.append(die)
        for item in self.desordem:
            state.append(self.desordem[item])
        return state
        
    def set_state(self, state):
        self.rolls_left = state[0]
        index = 1
        for die in range(len(self.dices)):
            self.dices[die] = state[index]
            index += 1

        index = 6
        for item in self.desordem:
            self.desordem[item] = state[index]
            index += 1

        return self.get_game_state()


    # onde é possivel anotar o resultado dos dados
    def get_empty_cells(self, state):
        self.set_state(state)
        avaiable_list = {}
        for item in self.desordem:
            if self.desordem[item] == -1:
                avaiable_list[item] = self.desordem[item]
        return avaiable_list
    

    def get_number_of_actions(self, state):
        return 13
        if state[0] == Game_Moment.TABELA:
            # se estivermos na tabela, retornamos quantidade de celulas em branco
            return len(self.get_empty_cells(state))
        elif state[0] == Game_Moment.SECOS_1 or state[0] == Game_Moment.SECOS_2:
            #se tivermos no SECO retornamos binário "marcar/ não marcar"
            return 2
        else:
            # se estivermos nos dados retornamos 31 combinações de dados
            return 31


    def do_the_reroll(self, n_dices):
        dices_to_reroll = '{0:05b}'.format(int(n_dices))
        self.game_play.append("dados a serem rolados novamente:     " + dices_to_reroll)
        for index in range(len(str(dices_to_reroll))):
            if dices_to_reroll[index] == "1":
                self.dices[index] = self.roll_dice(1)[0]

        self.dices.sort()
        self.game_play.append("seus dados ficaram assim:            " + str(self.dices))
        dice_set = set(self.dices)
        n_single = len(dice_set)
        # return (6 - n_single) * 3
        return 0


    def reset(self):
        self.game_play = []
        self.game_play.append("Jogo iniciado")
        self.desordem = {"1": -1, "2": -1, "3": -1, "4": -1, "5": -1, "6": -1, "q": -1, "f": -1, "s+": -1, "s-": -1,
                         "x+": -1, "x-": -1,
                         "y": -1}
        self.dices = self.roll_dice(5)
        self.dices.sort()
        self.is_ended = False
        self.rolls_left = 2

        return self.get_game_state()
    

    def get_initial_state(self):
        return self.reset()
    

    def get_valid_moves(self, state):
        return np.ones(13)
        game_state = state[0]       
        if game_state == Game_Moment.TABELA:
            # se estivermos na tabela, retornamos quantidade de celulas em branco
            return np.ones(len(self.get_empty_cells(state)))
        elif game_state == Game_Moment.SECOS_1 or game_state == Game_Moment.SECOS_2:
            #se tivermos no SECO retornamos binário "marcar/ não marcar"
            return np.ones(2)
        else:
            # se estivermos nos dados retornamos 31 combinações de dados
            return np.ones(31)
        # return self.get_number_of_actions()
        # return (state.reshape(-1) == 0).astype(np.uint8)


    def is_full(self):
        return ((self.dices[0] == self.dices[2] and self.dices[3] == self.dices[4]) or (
                        self.dices[0] == self.dices[1] and self.dices[2] == self.dices[4]))


    def set_cell_value(self, cell: str):
        
        self.game_play.append("você marcou                          " + str(self.dices) + " em " + cell)
        self.game_play.append("\n")
        
        # self.marcado_em = 
        # print(self.marcado_em)
        points = 0

        if cell == "1":
            points = self.dices.count(1)
            self.desordem["1"] = points
            return points
        
        elif cell == "2":
            points = self.dices.count(2) *2
            self.desordem["2"] = points
            return points
        
        elif cell == "3":
            points = self.dices.count(3) * 3
            self.desordem["3"] = points
            return points
        
        elif cell == "4":
            points = self.dices.count(4) * 4
            self.desordem["4"] = points
            return points
        
        elif cell == "5":
            points = self.dices.count(5) * 5
            self.desordem["5"] = points
            return points
        
        elif cell == "6":
            points = self.dices.count(6) * 6
            self.desordem["6"] = points
            return points
        
        elif cell == "y":
            if self.dices[0] == self.dices[4]:
                self.desordem["y"] = sum(self.dices) + 50
                return sum(self.dices) + 50
            else:
                self.desordem["y"] = 0
                return 0
            
        elif cell == "q":
            if self.dices[0] == self.dices[3]:
                self.desordem["q"] = sum(self.dices[:4]) + 30
                return sum(self.dices[:4]) + 30
            elif self.dices[1] == self.dices[4]:
                self.desordem["q"] = sum(self.dices[1:]) + 30
                return sum(self.dices[1:]) + 30
            else:
                self.desordem["q"] = 0
                return 0
            
        elif cell == "f":
            if self.is_full():
                self.desordem["f"] = sum(self.dices) + 20
                return sum(self.dices) + 20

            else:
                self.desordem["f"] = 0
                return 0
            
        elif cell == "s+":
            if self.check_consecutive(self.dices) and self.dices[0] == 2:
                self.desordem["s+"] = 60
                return 60
            else:
                self.desordem["s+"] = 0
                return 0
            
        elif cell == "s-":
            if self.check_consecutive( self.dices) and self.dices[0] == 1:
                self.desordem["s-"] = 50
                return 50
            else:
                self.desordem["s-"] = 0
                return 0
            
        elif cell == "x+":
            if (sum(self.dices) > self.desordem["x-"]) or self.desordem["x-"] == -1:
                self.desordem["x+"] = sum(self.dices)
                return sum(self.dices)
            else:
                self.desordem["x+"] = 0
            return 0
        
        elif cell == "x-":
            if (sum(self.dices) < self.desordem["x+"]) or self.desordem["x+"] == -1:
                self.desordem["x-"] = sum(self.dices)
                return sum(self.dices)
            else:
                self.desordem["x-"] = 0
            return 0
            

    def go_for_n(self, n):
        n_count = self.dices.count(n)
        if n_count == 5 or self.rolls_left == 0:
            self.rolls_left = 2
            reward = self.set_cell_value(str(n))
            self.dices = self.roll_dice(5)
            self.dices.sort()
            return reward
        else:
            for index in range(5):
                if self.dices[index] != n:
                    self.dices[index] = self.roll_dice(1)[0]

        self.rolls_left -= 1
        self.dices.sort()
        return 0


    def go_for_y(self):
        dices_count = []
        for n in range (1,7):
            dices_count.append(self.dices.count(n))
        if np.max(dices_count) > 4 or self.rolls_left == 0:
            self.rolls_left = 2
            reward = self.set_cell_value("y")
            self.dices = self.roll_dice(5)
            self.dices.sort()
            return reward

        if dices_count.count(np.max(dices_count)) > 1:
            most_dices_indexs = []
            most_dices = None
            for index in range(len(dices_count)):
                if dices_count[index] == np.max(dices_count):
                    most_dices_indexs.append(index + 1)
            
            #checar qual dado manter dependendo da tabela
            for die in most_dices_indexs:
                if self.desordem[str(die)] == -1:
                    most_dices = die

            if most_dices == None:
                most_dices = most_dices_indexs[len(most_dices_indexs)-1]
        else:        
            most_dices = np.argmax(dices_count) + 1

        a=1

        for index in range(5):
            if self.dices[index] != most_dices:
                self.dices[index] = self.roll_dice(1)[0]

        self.dices.sort()
        self.rolls_left -= 1
        return 0


    def go_for_q(self):
        dices_count = []
        for n in range (1,7):
            dices_count.append(self.dices.count(n))
        if np.max(dices_count) > 3 or self.rolls_left == 0:
            self.rolls_left = 2
            reward = self.set_cell_value("q")
            self.dices = self.roll_dice(5)
            self.dices.sort()
            return reward

        if dices_count.count(np.max(dices_count)) > 1:
            most_dices_indexs = []
            most_dices = None
            for index in range(len(dices_count)):
                if dices_count[index] == np.max(dices_count):
                    most_dices_indexs.append(index + 1)
            
            #checar qual dado manter dependendo da tabela
            for die in most_dices_indexs:
                if self.desordem[str(die)] == -1:
                    most_dices = die

            if most_dices == None:
                most_dices = most_dices_indexs[len(most_dices_indexs)-1]
        else:        
            most_dices = np.argmax(dices_count) + 1

        for index in range(5):
            if self.dices[index] != most_dices:
                self.dices[index] = self.roll_dice(1)[0]

        self.dices.sort()
        self.rolls_left -= 1
        return 0


    def go_for_f(self):
        dices_count = []
        for n in range (1,7):
            dices_count.append(self.dices.count(n))
        if self.is_full() or self.rolls_left == 0:
            self.rolls_left = 2
            reward = self.set_cell_value("f")
            self.dices = self.roll_dice(5)
            self.dices.sort()
            return reward

        if dices_count.count(np.max(dices_count)) == 2:
            least_dices = None
            for index in range(len(dices_count)):
                if dices_count[index] == 1:
                    least_dices = (index + 1)
            for index in range(5):
                if self.dices[index] == least_dices:
                    self.dices[index] = self.roll_dice(1)[0]

        elif dices_count.count(np.max(dices_count)) == 1:
            most_dices = np.argmax(dices_count) +1
            for index in range(5):
                if self.dices[index] != most_dices:
                    self.dices[index] = self.roll_dice(1)[0]

        elif dices_count.count(np.max(dices_count)) > 2:
            most_dices = None
            most_dices_indexs = []
            for index in range(len(dices_count)):
                if dices_count[index] == np.max(dices_count):
                    most_dices_indexs.append(index + 1)
            
            #checar qual dado manter dependendo da tabela
            for die in most_dices_indexs:
                if self.desordem[str(die)] == -1:
                    most_dices = die

            if most_dices == None:
                most_dices = most_dices_indexs[len(most_dices_indexs)-1]

            for index in range(5):
                if self.dices[index] != most_dices:
                    self.dices[index] = self.roll_dice(1)[0]

        self.dices.sort()
        self.rolls_left -= 1
        return 0


    def go_for_s(self, witch_s):
        dices_count = []
        for n in range (1,7):
            dices_count.append(self.dices.count(n))
        if self.rolls_left == 0:
            self.rolls_left = 2
            reward = self.set_cell_value(witch_s)
            self.dices = self.roll_dice(5)
            self.dices.sort()
            return reward
        
        if self.check_consecutive(self.dices) and self.dices[0] == 2 and witch_s == "s+":
            self.rolls_left = 2
            reward = self.set_cell_value("s+")
            self.dices = self.roll_dice(5)
            self.dices.sort()
            return reward
        
        if self.check_consecutive(self.dices) and self.dices[0] == 1 and witch_s == "s-":
            self.rolls_left = 2
            reward = self.set_cell_value("s-")
            self.dices = self.roll_dice(5)
            self.dices.sort()
            return reward
        
        reroll_dices = []

        if witch_s == "s+":
            for n in range(dices_count[0]):
                reroll_dices.append(1)
                
            for index in range(1, 6):
                for quant in range(dices_count[index] - 1):
                    reroll_dices.append(index + 1)

        else:
            for n in range(dices_count[5]):
                reroll_dices.append(6)

            for index in range(5):
                for quant in range(dices_count[index] - 1):
                    reroll_dices.append(index + 1)

        reroll_dices.sort()

        for index in range(5):
            if self.dices[index] in reroll_dices:
                reroll_dices = np.delete(reroll_dices, 0)
                self.dices[index] = self.roll_dice(1)[0]

        self.rolls_left -= 1
        self.dices.sort()
        return 0


    def get_next_state(self, state, action):
        self.set_state(state)
        action = int(action)


        if action < 6:
            self.reward = self.go_for_n(action+1)
        elif action == 6:
            self.reward = self.go_for_q()
        elif action == 7:
            self.reward = self.go_for_f()
        elif action == 8:
            self.reward = self.go_for_s("s+")
        elif action == 9:
            self.reward = self.go_for_s("s-")
        elif action == 10:
            self.rolls_left = 2
            self.reward = self.set_cell_value("x+")
            self.dices = self.roll_dice(5)
            self.dices.sort()
        elif action == 11:
            self.rolls_left = 2
            self.reward = self.set_cell_value("x-")
            self.dices = self.roll_dice(5)
            self.dices.sort()
        elif action == 12:
            self.reward = self.go_for_y()
        else:
            breakpoint()
        
        return (self.get_game_state())


    def check_ended(self, state):
        self.set_state(state)
        return -1 not in self.desordem.values()
    

    def get_value_and_terminated(self, state, action):
        if action == None:
            return (0, False)
        self.set_state(state)
        if self.check_ended(state):
            return self.get_total_score(state), self.check_ended(state)
        else:
            return self.reward, self.check_ended(state)
    

    def get_total_score(self, state):
        self.set_state(state)
        self.score_values = list(self.desordem.values())
        self.total = sum(self.score_values)
        total_upper = sum(self.score_values[0:6])
        if total_upper >= 60:
            self.total += 30
        # print()
        self.game_play.append("sua tabela ficou assim:              " + str(self.desordem))
        self.game_play.append("TOTAL:                               " + str(self.total))
        print("total: ", self.total)
        print("sua tabela ficou assim: ", self.desordem)
        print()
        

        return self.total
    

    def get_encoded_state(self, state):
        encoded = state.copy()
        encoded[0] = state[0].value
        encoded = np.stack(encoded).astype(np.float32)
        return encoded



In [ ]:


game = Yan()
state = game.get_initial_state()
while True:
    print(f"rerolls: {game.rolls_left} | Dados: {state[1:6]} | Tabela: {game.desordem}")
    time.sleep(1)
    
    user_input = input()
    action = None
    if user_input == "1":
        action = 0
    if user_input == "2":
        action = 1
    if user_input == "3":
        action = 2
    if user_input == "4":
        action = 3
    if user_input == "5":
        action = 4
    if user_input == "6":
        action = 5
    if user_input == "q":
        action = 6
    if user_input == "f":
        action = 7
    if user_input == "s+":
        action = 8
    if user_input == "s-":
        action = 9
    if user_input == "x+":
        action = 10
    if user_input == "x-":
        action = 11
    if user_input == "y":
        action = 12
    state = game.get_next_state(state, action)
    if game.check_ended(state):
        print(f"Total: {game.get_total_score(state)}")
        print()
        print()
        print()

        game.reset()
        state = game.get_initial_state()

    else:
        print(f"Reward: {game.reward}")
        print()


In [ ]:
NUM_CLASSES = 13
NUM_FEATURES = 19
RANDOM_SEED = 42


In [8]:

device  = "cuda" if torch.cuda.is_available() else "cpu"


class BobNet(nn.Module):
    def __init__(self, input_features=NUM_FEATURES, output_features=NUM_CLASSES, hiden_units = 32, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.policyHead = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hiden_units),
            nn.ReLU(),
            nn.Linear(in_features=hiden_units, out_features=hiden_units),
            nn.ReLU(),
            nn.Linear(in_features=hiden_units, out_features=output_features)
        )
        
        self.valueHead = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hiden_units),
            nn.ReLU(),
            nn.Linear(in_features=hiden_units, out_features=hiden_units),
            nn.ReLU(),
            nn.Linear(in_features=hiden_units, out_features=1)
        )
        

    def forward(self, x):
        policy = self.policyHead(x)
        value = self.valueHead(x)
        return policy, value
        
policy_model = BobNet().to(device)
policy_model


NameError: name 'torch' is not defined

In [ ]:
class ResNet(nn.Module):
    def __init__(self, game, num_resBlocks, num_hidden):
        super().__init__()
        self.startBlock = nn.Sequential(
            nn.Linear(19, num_hidden),
            nn.BatchNorm1d(num_hidden),
            nn.ReLU()
        )
        
        self.backBone = nn.ModuleList(
            [ResBlock(num_hidden) for i in range(num_resBlocks)]
        )
        
        self.policyHead = nn.Sequential(
            nn.Linear(num_hidden, 31),
            nn.BatchNorm1d(31),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(31 * 19, 31)
        )
        
        self.valueHead = nn.Sequential(
            nn.Linear(num_hidden, 3),
            nn.BatchNorm1d(3),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(19, 2),
            nn.Tanh()
        )
        
    def forward(self, x):
        x = self.startBlock(x)
        for resBlock in self.backBone:
            x = resBlock(x)
        policy = self.policyHead(x)
        value = self.valueHead(x)
        return policy, value
    
class ResBlock(nn.Module):
    def __init__(self, num_hidden):
        super().__init__()
        self.conv1 = nn.Linear(num_hidden, num_hidden)
        self.bn1 = nn.BatchNorm1d(num_hidden)
        self.conv2 = nn.Linear(num_hidden, num_hidden)
        self.bn2 = nn.BatchNorm1d(num_hidden)
        
    def forward(self, x):
        residual = x
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        x = F.relu(x)
        return x

In [ ]:
import matplotlib.pyplot as plt

yan = Yan()

state = yan.get_initial_state()
# state = yan.get_next_state(state, 2, 1)
# state = yan.get_next_state(state, 7, -1)

# print(state)

encoded_state = yan.get_encoded_state(state)

# print(encoded_state)

tensor_state = torch.tensor(encoded_state).unsqueeze(0)
model = BobNet().to(device)
model.eval()

policy, value = model(tensor_state)
value = value.item()
policy = torch.softmax(policy, axis=1).squeeze(0).detach().cpu().numpy()

print(value, policy)

plt.bar(range(NUM_CLASSES), policy)
plt.show()


In [ ]:
class Node:
    def __init__(self, game, args, state, parent=None, action_taken=None, prior=0):
        self.game = game
        self.args = args
        self.state = state
        self.parent = parent
        self.action_taken = action_taken
        self.prior = prior
        
        self.children = []
        
        self.visit_count = 0
        self.value_sum = 0
        
    def is_fully_expanded(self):
        return len(self.children) > 0
    
    def select(self):
        best_child = None
        best_ucb = -np.inf
        
        for child in self.children:
            ucb = self.get_ucb(child)
            if ucb > best_ucb:
                best_child = child
                best_ucb = ucb
                
        return best_child
    
    def get_ucb(self, child):
        if child.visit_count == 0:
            q_value = 0
        else:
            q_value = 1 - ((child.value_sum / child.visit_count) + 1) / 2
        return q_value + self.args['C'] * (math.sqrt(self.visit_count) / (child.visit_count + 1)) * child.prior
    
    def expand(self, policy):
        for action, prob in enumerate(policy):
            if prob > 0:
                child_state = self.state.copy()
                child_state = self.game.get_next_state(child_state, action)

                child = Node(self.game, self.args, child_state, self, action, prob)
                self.children.append(child)
                
        return child
    
            
    def backpropagate(self, value):
        self.value_sum += value
        self.visit_count += 1
        
        value = self.game.reward
        if self.parent is not None:
            self.parent.backpropagate(value)  


class MCTS:
    def __init__(self, game:Yan, args, model):
        self.game = game
        self.args = args
        self.model = model

    @torch.no_grad() 
    def search(self, state):
        root = Node(self.game, self.args, state)
        
        for search in range(self.args['num_searches']):
            node = root
            
            while node.is_fully_expanded():
                node = node.select()
                
            value, is_terminal = self.game.get_value_and_terminated(node.state, node.action_taken)
            
            if not is_terminal:
                policy, value = self.model(
                    torch.tensor(self.game.get_encoded_state(node.state)).unsqueeze(0)
                )   
                policy = torch.softmax(policy, axis=1).squeeze(0).cpu().numpy()
                # valid_moves = self.game.get_valid_moves(node.state)
                # valid_sized_move = valid_moves + np.zeros(13)
                # policy *= valid_sized_move
                # policy /= np.sum(policy)
                
                value = value.item()
                
                node.expand(policy)
                
            node.backpropagate(value)    
            
            
        action_probs = self.game.get_valid_moves(state)
        for child in root.children:
            action_probs[child.action_taken] = child.visit_count
        action_probs /= np.sum(action_probs)
        return action_probs
        
        
        

In [ ]:
game = Yan()

args = {
    'C': 2,
    'num_searches': 100
}
model = BobNet()
model.eval()
mcts = MCTS(game, args, model)

state = game.get_initial_state()


while True:
    # print(state)
    
    mcts_probs = mcts.search(state)
    action = np.argmax(mcts_probs)
        
    state = game.get_next_state(state, action)
    
    value, is_terminal = game.get_value_and_terminated(state, action)
    
    if is_terminal:
        print(game.desordem)
        print(game.get_total_score(state))
        game.reset()
        break
        
    # player = game.get_opponent(player)
